In [1]:
import pandas as pd
import numpy as np
import requests
def get_solved_problems_in_rating_range(user_handle, min_rating, max_rating):
    """Gets the set of problems done by a Codeforces user within a specific rating range.

    Args:
        user_handle: The handle of the user.
        min_rating: The minimum rating (inclusive) for the desired range.
        max_rating: The maximum rating (inclusive) for the desired range.

    Returns:
        A set of problem IDs.
    """
    solved_problems = set()
    page = 1

    while True:
        response = requests.get(
            f"https://codeforces.com/api/user.status?handle={user_handle}&from=1&count=10000&page={page}")

        data = response.json()

        if data["status"] != "OK":
            break

        for submission in data["result"]:
            if submission["verdict"] == "OK" and "problem" in submission:
                problem = submission["problem"]
                if "rating" in problem:
                    rating = problem["rating"]
                    if min_rating <= rating <= max_rating:
                        solved_problems.add((problem["contestId"], problem["index"]))

        if len(data["result"]) < 10000:
            break  

        page += 1

    return solved_problems

df = pd.read_csv('chunk_3.csv')

min_rating = 1300
max_rating = 1500

solved_problems_list = []
problem_count_list = []

for index, row in df.iterrows():
    user_handle = row["Handle"] 
    solved_problems = get_solved_problems_in_rating_range(user_handle, min_rating, max_rating)
    solved_problems_list.append(solved_problems)
    problem_count_list.append(len(solved_problems))  

df["Solved Problems"] = solved_problems_list
df["Problem Count"] = problem_count_list

df.to_csv("updated_chunk3.csv", index=False)